## 3 Welcom to Langchain

In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You vre a geography expertc. And you only reply in {language}"),
    AIMessage(content="Hi my name is {name}!"),
    HumanMessage(content="서울과 도쿄의 거리는 얼마인가요. 너의 이름은 뭐니?"),
]

chat.predict_messages(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language = "Korean",
    name = "AiGENDA",
    country_a = "Korea",
    country_b = "USA"
)

chat.predict_messages(prompt).content

### 3.3 OutputParser and LCEL 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    name = "aigenda",
    language = "Korean",
    max_items = 10,
    question = "2024 한국 축구 국가대표 맴버",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()    

chain.invoke({
    "name":"aigenda",
    "language":"Korean",
    "max_items":5,
    "question":"2024 한국 축구 국가대표 맴버",
})

### 3.4 Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. \
         Everything you are asked will be answered with a comma separated list of max 5 in lowercase. \
         Do NOT reply with anything else."),
        # ("human", "{question}"),
        ("human", "I want to cook {cuisine} food."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. \
         You find alternative ingredients and explain their preparation. \
         You don't radically modify the recipe. \
         If there is no alternative for a food just say you don't know how to replace it. \
         And you only reply in Korean"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_prompt | chat

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine":"korea"})

